In [1]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import pandas as pd
# os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [4]:
df = pd.read_csv('./graph_20211207.csv')
_l = list(zip(list(df['d_name']), list(df['did'])))
_l.extend(list(zip(list(df['e_name']), list(df['eid']))))
name2entity = {str(item[0]).strip(): str(item[1]).strip() for item in _l}

entity2name = {v: k for k, v in name2entity.items()}

entity2id = {}
with open('./entity2id.txt', 'r', encoding='utf8') as f:
    for line in f:
        line = line.strip()
        arr = line.split('\t')
        if len(arr) == 2:
            entity_name = arr[0].strip()
            entity_id = int(arr[1].strip())
            if entity_name not in entity2id:
                entity2id[entity_name] = entity_id
                
with open('./triplets.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()
    
conditions = []
for line in lines:
    arr = line.strip().split(',')
    h = arr[0].strip()
    t = arr[1].strip()
    conditions.append(entity2id[h])
        
conditions = list(set(conditions))

all_disease = conditions.copy()
n_disease = len(all_disease)

did2dindex = {}
with open('./did2dindex.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
        arr = line.strip().split(':')
        did = int(arr[0])
        dindex = int(arr[1])
        if did not in did2dindex:
            did2dindex[did] = int(dindex)        
dindex2did = {v: k for k, v in did2dindex.items()}

entity2id = {}
with open('./entity2id.txt', 'r', encoding='utf8') as f:
    for line in f:
        line = line.strip()
        arr = line.split('\t')
        if len(arr) == 2:
            entity_name = arr[0].strip()
            entity_id = int(arr[1].strip())
            if entity_name not in entity2id:
                entity2id[entity_name] = entity_id
id2entity = {v: k for k, v in entity2id.items()}
input_dimension = len(entity2id)


# openKE上训练，本地离线训练，有tf和pytorch版本算法包（没有预训练模型），联合训练，输入数据：三元组
f = open('./model_n_matrix.pkl', 'rb')
model_n_matrix = pickle.load(f)
f.close()

f = open('./model_r_matrix.pkl', 'rb')
model_r_matrix = pickle.load(f)
f.close()

model_n_matrix = torch.tensor(model_n_matrix).to(device)
model_r_matrix = torch.tensor(model_r_matrix).to(device)

model_n_matrix.requires_grad = False
model_r_matrix.requires_grad = False

def convert(input_list):
    node_id_list = input_list

    #nodes [1 * V]
    nodes = sindex2tensor(node_id_list)

    return nodes

def sindex2tensor(idxs):
    x = np.zeros(input_dimension)
    for idx in idxs:
        x[int(idx)] = 1
    return to_tensor([x])

def to_tensor(x):
    return torch.tensor(x, dtype=torch.float).to(device)

class Net(nn.Module):
    '''
    初始化网络
    '''
    def __init__(self, n_disease, input_dimension):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dimension, 150)
        self.linear2 = nn.Linear(150, n_disease)
        self.sigmoid = nn.Sigmoid()

    '''
    前向计算过程
    '''
    def forward(self, x):
        x = F.relu(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        #x = F.softmax(x, dim=2)
        tag_space = self.sigmoid(x) #多标签拼binary cross entropy
        return tag_space
    
model = Net(n_disease, 150).to(device)
model.load_state_dict(torch.load('./Diagnose_net_20211207_with_mutant_2999_cpu.pkl'))
model.eval()

## complementary
df_complementary = pd.read_excel('complementary.xlsx')

filename=[Content_Types].xml
self.create_system=3
filename=_rels/
self.create_system=3
filename=_rels/.rels
self.create_system=3
filename=docProps/
self.create_system=3
filename=docProps/app.xml
self.create_system=3
filename=docProps/core.xml
self.create_system=3
filename=docProps/custom.xml
self.create_system=3
filename=xl/
self.create_system=3
filename=xl/_rels/
self.create_system=3
filename=xl/_rels/workbook.xml.rels
self.create_system=3
filename=xl/sharedStrings.xml
self.create_system=3
filename=xl/styles.xml
self.create_system=3
filename=xl/theme/
self.create_system=3
filename=xl/theme/theme1.xml
self.create_system=3
filename=xl/workbook.xml
self.create_system=3
filename=xl/worksheets/
self.create_system=3
filename=xl/worksheets/_rels/
self.create_system=3
filename=xl/worksheets/_rels/sheet1.xml.rels
self.create_system=3
filename=xl/worksheets/sheet1.xml
self.create_system=3
fname_str=xl/_rels/workbook.xml.rels
fname_str=xl/workbook.xml
fname_str=docProps/core.xml
fname_str=xl/sty

In [6]:
input_dimension

2195

In [5]:
top = 10
symptoms = ['跛行步态', '臀部疼痛', '肌张力降低', '脊柱侧弯', '腱反射消失', '神经痛']
entities = [name2entity[symptom] for symptom in symptoms if symptom in name2entity]
eids = [entity2id[entity] for entity in entities if entity in entity2id]

if len(eids) > 0:
    samples_symptoms = [eids]
    nodes_list = torch.stack([convert(samples_symptoms[0])])
    n_matrix = model_n_matrix.repeat(1, 1, 1)
    samples_tensors = nodes_list.bmm(n_matrix).squeeze(1)

    y_pred = model(samples_tensors)
    
    y_sorted = y_pred[0].sort()

    prob_list = list(y_sorted[0].detach().cpu().numpy())
    item_list = list(y_sorted[1].detach().cpu().numpy())

    prob_list.reverse()
    prob_list = prob_list[0: top]

    item_list.reverse()
    item_list = item_list[0: top]
    
    dindexs = [dindex2did[y] for y in item_list]
    dentities = [id2entity[did]  for did in dindexs]
    dnames = [entity2name[e]  for e in dentities]
    
    y_list = list(zip(dnames, prob_list))
    
    result = []
    for y in y_list:
        dname = y[0]
        subdf = df_complementary[df_complementary['d_name'] == dname]
        
        major_symptom = list(subdf[subdf['relation'] == 'has symptom_main']['e_name'])
        regular_symptom = list(subdf[subdf['relation'] == 'has symptom']['e_name'])
        examination_suggest = list(subdf[subdf['relation'] == 'examination suggest']['e_name'])
        medication_suggest = list(subdf[subdf['relation'] == 'medication suggest']['e_name'])
        
        _result = dict()
        _result['suggested_disease'] = y
        _result['major_symptom'] = major_symptom
        _result['regular_symptom'] = regular_symptom
        _result['examination_suggest'] = examination_suggest
        _result['medication_suggest'] = medication_suggest
        result.append(_result)
result

[{'suggested_disease': ('坐骨神经痛', 0.99999833),
  'major_symptom': ['跛行步态', '臀部疼痛'],
  'regular_symptom': ['肌张力降低', '脊柱侧弯', '腱反射消失', '神经痛'],
  'examination_suggest': ['腰椎MRI平扫', '腰椎X线正位片', '直腿抬高试验'],
  'medication_suggest': ['阿司匹林', '利多卡因', '维生素B12']},
 {'suggested_disease': ('神经性贪食症', 0.35877183),
  'major_symptom': ['多饮', '缺乏饱食感'],
  'regular_symptom': ['贪食', '体重增加'],
  'examination_suggest': [],
  'medication_suggest': ['奥沙西泮', '帕罗西汀']},
 {'suggested_disease': ('低钙血症', 0.31086078),
  'major_symptom': ['四肢抽搐'],
  'regular_symptom': ['癫痫发作',
   '喉部痉挛',
   '幻觉',
   '小支气管痉挛',
   '心律失常',
   '骨骼畸形',
   '骨痛',
   '睡觉时小腿抽筋',
   '谵妄'],
  'examination_suggest': ['血清钙', '磷 (Phosphorus)P', '心电图'],
  'medication_suggest': ['葡萄糖酸钙']},
 {'suggested_disease': ('慢性闭角型青光眼', 0.2654089),
  'major_symptom': ['头痛', '虹视', '眼痛', '眼压升高', '眼胀'],
  'regular_symptom': ['青光眼斑'],
  'examination_suggest': ['眼底荧光血管造影（FFA）',
   '眼前节OCT',
   '视野检查',
   '眼压测量',
   '裂隙灯',
   '前房角镜检查'],
  'medication_suggest': ['噻吗洛尔', '毛